In [19]:
# !pip install psutil

In [22]:
import psutil
my_memo = psutil.virtual_memory()
print('Used memory:',round(my_memo[-2]/1024**3,1),'Gb')
print('Free memory:',round(my_memo[-1]/1024**3,1),'Gb')

Used memory: 3.3 Gb
Free memory: 4.6 Gb


In [23]:
import numpy as np
from keras.utils import to_categorical as keras_ohe
import tensorflow as tf
from tensorflow import one_hot as tensor_ohe

arr = np.array([1,2,3,4,5,5,5,8,2,1,77]*1000000, dtype='int32')
uniq = np.unique(arr).tolist()

tf_arr = tf.constant(arr, dtype='int32')
tf_uniq = tf.constant(len(uniq), dtype='int32')

def numpy_ohe_1(x):
    unique_list = np.unique(arr).tolist()
    return np.array([(x == i)*1 for i in unique_list]).T

def numpy_ohe_2(x,unique_list):
    return np.array([(x == i)*1 for i in unique_list]).T

def tf_ohe_inside(arr,uniq):
    sess = tf.InteractiveSession() 
    tf_arr = tf.Variable(arr)
    tf_uniq = tf.Variable(uniq)
    sess.run(tf.global_variables_initializer())
    ohe = tensor_ohe(tf_arr, tf_uniq[0]).eval()
    sess.close()
    return ohe

Using TensorFlow backend.


In [24]:
my_memo = psutil.virtual_memory()
print('Used memory:',round(my_memo[-2]/1024**3,1),'Gb')
print('Free memory:',round(my_memo[-1]/1024**3,1),'Gb')

Used memory: 3.5 Gb
Free memory: 4.4 Gb


In [25]:
sess = tf.InteractiveSession() 

In [26]:
%%timeit
numpy_ohe_1(arr) 
# функция с numpy (вычислением количества категорий внутри функции
# каждый раз - время - долго!)

648 ms ± 9.93 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
%%timeit
numpy_ohe_2(arr,uniq)
# функция с numpy (вычисление количества категорий вынесено за пределы фукции
# один раз - время - быстрее)

411 ms ± 18.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%%timeit
keras_ohe(arr)
# реализация one-hot-encoding в keras

1.61 s ± 28.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
%%timeit
tensor_ohe(arr, len(uniq)).eval()
# реализация на тензорах - на вход подается numpy-вектор и число

4.57 s ± 302 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
%%timeit
tensor_ohe(tf_arr, tf_uniq).eval()
# реализация на тензорах - на вход подается tensor-вектор и tensor-число
# инициализация константами

4.03 s ± 55.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
tf_v_arr = tf.Variable(arr)
tf_v_uniq = tf.Variable(uniq)
sess.run(tf.global_variables_initializer())

In [32]:
%%timeit
tensor_ohe(tf_v_arr, tf_v_uniq[0]).eval()
# реализация на тензорах - иниициализация один раз - время - не самое хорошее
# инициализация переменной variable (до выполнения графа)

1.51 s ± 45.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
tf_p_arr = tf.placeholder(dtype='int32', shape=arr.shape)
tf_p_uniq = tf.placeholder(dtype='int32', shape=())
tf_p_ohe = tensor_ohe(tf_p_arr, tf_p_uniq)

In [34]:
%%timeit
tf_p_ohe.eval({tf_p_arr: arr, tf_p_uniq: len(uniq)})
# реализация на тензорах - на вход подаются параметры - НЕ тензоры 
# с предварительной резервацией плейсхолдеров - время лучшее, сопостовимое с numpy

554 ms ± 4.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
sess.close()

In [36]:
%%timeit
tf_ohe_inside(arr,uniq)
# реализация на тензорах - инициализация каждый раз - время - отвратительное 
# инициализация переменной tensor variable - внутри функции - очевидно, неправильно!

6.12 s ± 782 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
my_memo = psutil.virtual_memory()
print('Used memory:',round(my_memo[-2]/1024**3,1),'Gb')
print('Free memory:',round(my_memo[-1]/1024**3,1),'Gb')

Used memory: 3.3 Gb
Free memory: 4.6 Gb
